# Processando Casos e Óbitos

Neste notebook voc6e encontra os scripts de processamento dos dados de casos e  óbitos do Estado de Sergipe. Os dados foram coletados direto do site do [https://brasio.io](brasil.io) que coleta, organiza e publica dados de casos e óbitos de todo o Brasil a nível de município. Os dados utilizados aqui são apenas um recorte dos dados completos filtrando pelo estado de Sergipe. O site atualiza diariamente os dados.

Neste caso, não foi feito nenhum tipo de processamento mais amplo nos dados já que eles já passaram por isso no site do Brasil.io. Aqui apenas coletamos os dados específico de Sergipe, renomeamos as colunas e calculamos a média móvel de 7 dias para cada um dos municípios. Essa base foi preparada para facilitar a visualização das curvas de casos e óbitos em visualizações. 

## Imports necessários

In [51]:
import pandas as pd

## Leitura dos dados de entrada

O arquivo `casos_obitos_se.csv` foi baixado direto [desta página](https://brasil.io/dataset/covid19/caso_full/) do site do brasil.io. 

In [52]:
casos_obitos = pd.read_csv("../input/casos_obitos_se.csv")
casos_obitos

,epidemiological_week,date,order_for_place,state,city,city_ibge_code,place_type,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,new_confirmed,last_available_deaths,new_deaths,last_available_death_rate,estimated_population,is_last,is_repeated
0,202145,2021-11-13,553,SE,Amparo de São Francisco,2800100,city,256,10756.30252,0,5,0,0.0195,2380,False,True
1,202145,2021-11-13,565,SE,Aquidabã,2800209,city,2409,11111.11111,0,40,0,0.0166,21681,False,True
2,202145,2021-11-13,610,SE,Aracaju,2800308,city,127885,19233.48794,0,2414,0,0.0189,664908,False,True
3,202145,2021-11-13,561,SE,Arauá,2800407,city,1292,12988.84086,0,22,0,0.0170,9947,False,True
4,202145,2021-11-13,566,SE,Areia Branca,2800506,city,2104,11259.76667,0,48,0,0.0228,18686,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42734,202012,2020-03-16,3,SE,NaN,28,state,5,0.21563,4,0,0,0.0000,2318822,False,False
42735,202012,2020-03-15,2,SE,Aracaju,2800308,city,1,0.15040,0,0,0,0.0000,664908,False,True
42736,202012,2020-03-15,2,SE,NaN,28,state,1,0.04313,0,0,0,0.0000,2318822,False,True
42737,202011,2020-03-14,1,SE,Aracaju,2800308,city,1,0.15040,1,0,0,0.0000,664908,False,False


## Filtrando colunas

Como o objetivo é disponibilizar apenas informaçòes de casos e óbitos, apenas algumas colunas foram selecionadas para o arquivo final. 

In [53]:
data_final = casos_obitos.query("place_type == 'city'")
data_final = data_final.sort_values(by=['city','date'])
data_final = data_final[['date','city_ibge_code','city','last_available_confirmed','new_confirmed','last_available_deaths','new_deaths']]
data_final = data_final.reset_index(drop=True)
data_final

,date,city_ibge_code,city,last_available_confirmed,new_confirmed,last_available_deaths,new_deaths
0,2020-05-10,2800100,Amparo de São Francisco,1,1,0,0
1,2020-05-11,2800100,Amparo de São Francisco,1,0,0,0
2,2020-05-12,2800100,Amparo de São Francisco,1,0,0,0
3,2020-05-13,2800100,Amparo de São Francisco,2,1,0,0
4,2020-05-14,2800100,Amparo de São Francisco,2,0,0,0
...,...,...,...,...,...,...,...
42124,2021-11-09,2807600,Umbaúba,2134,0,41,0
42125,2021-11-10,2807600,Umbaúba,2134,0,41,0
42126,2021-11-11,2807600,Umbaúba,2134,0,41,0
42127,2021-11-12,2807600,Umbaúba,2134,0,41,0


## Cálculo da média móvel

Nesse trecho, é calculada a média móvel para casos e óbitos. Essa informação é calculada a partir do quantitativo diário de óbitos e casos. Utilizou-se 7 dias de intervalo. Ou seja, a média de um dia é baseada nos dados do dia atual e dos 6 dias ateriores, totalizando 7 dias. 

In [54]:
all_codes = list(set(data_final.city_ibge_code))

In [55]:
for code in all_codes:
    data_final.loc[data_final['city_ibge_code'] == code, 'rolling_7_confirmed'] = data_final.loc[data_final['city_ibge_code'] == code, 'new_confirmed'].rolling(window=7).mean()
    data_final.loc[data_final['city_ibge_code'] == code, 'rolling_7_deaths'] = data_final.loc[data_final['city_ibge_code'] == code, 'new_deaths'].rolling(window=7).mean()


## Gerando o arquivo final

O arquivo final é salvo na pasta `output` no formato `.csv`. Para manter o mesmo padrão de colunas das demais bases deste repositório, as colunas foram renomeadas. 

In [56]:
data_final = data_final.rename(columns={
    'date': 'data', 
    'city_ibge_code': 'codigo_municipio', 
    'city': 'nome_municipio',
    'last_available_confirmed': 'casos_confirmados_total',
    'new_confirmed': 'casos_confirmados_diarios',
    'last_available_deaths': 'obitos_confirmados_total',
    'new_deaths': 'obitos_confirmados_diarios',
    'rolling_7_confirmed': 'casos_media_movel',
    'rolling_7_deaths': 'obitos_media_movel'
    }
)
data_final

,data,codigo_municipio,nome_municipio,casos_confirmados_total,casos_confirmados_diarios,obitos_confirmados_total,obitos_confirmados_diarios,casos_media_movel,obitos_media_movel
0,2020-05-10,2800100,Amparo de São Francisco,1,1,0,0,NaN,NaN
1,2020-05-11,2800100,Amparo de São Francisco,1,0,0,0,NaN,NaN
2,2020-05-12,2800100,Amparo de São Francisco,1,0,0,0,NaN,NaN
3,2020-05-13,2800100,Amparo de São Francisco,2,1,0,0,NaN,NaN
4,2020-05-14,2800100,Amparo de São Francisco,2,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
42124,2021-11-09,2807600,Umbaúba,2134,0,41,0,0.0,0.0
42125,2021-11-10,2807600,Umbaúba,2134,0,41,0,0.0,0.0
42126,2021-11-11,2807600,Umbaúba,2134,0,41,0,0.0,0.0
42127,2021-11-12,2807600,Umbaúba,2134,0,41,0,0.0,0.0


In [57]:
data_final.to_csv("../output/municipio_casos_obitos.csv", index=False)